# <font color='blue'>Laboratório de Inovação ICTB - Instituto de Ciência e Tecnologia em Biomodelos</font>

## <font color='blue'>Projeto - Medição do Risco de Descarte Animal</font>

## Definição do Problema

Medição do risco de descarte animal de acordo com 2 linhagens e 3 níveis de classificação: baixo, médio e alto.

## Fonte de Dados

FIOCRUZ / ICTB

## Pacotes instalados

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Imports
import pickle
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "ICTB" --iversions

## Carregamento dos Dados

In [ ]:
# Download dos dados
df_black = pd.read_csv("dados/black6.csv", 
                     sep = ";")

In [ ]:
# Visualiza os dados
df_black.head()

In [ ]:
# Download dos dados
df_swiss = pd.read_csv("dados/swiss.csv",
                       sep = ";")

In [ ]:
# Visualiza os dados
df_swiss.head()

Merge (concatenação) dos datasets.

In [ ]:
# Criar novo atributo para representar a linhagem
df_black['linhagem'] = 0
df_swiss['linhagem'] = 1

In [ ]:
# Concatenação
df_linhagens = pd.concat([df_black, df_swiss], ignore_index = True)

In [ ]:
# Visualiza dados
df_linhagens.head()

In [ ]:
# Shape
df_linhagens.shape

## Análise Exploratória

In [ ]:
# Tipos dos dados
df_linhagens.info()

In [ ]:
#  Resumo estatístico
df_linhagens.describe()

In [ ]:
# Distribuição de classe
df_linhagens['risco'].value_counts()

In [ ]:
df_linhagens['risco'].hist(color = "red")
plt.suptitle('Histograma da Pontuação do Risco de Descarte')
plt.xlabel('Pontuação')
plt.ylabel('Número de Classificações')
plt.show()

## Estratégias de Balanceamento de Classe

In [ ]:
# Extrai as classes atuais
pontuacoes = set(df_linhagens['risco'])
pontuacoes

In [ ]:
# Vamos extrair 500 amostras de cada classe (undersampling)
df_temp = []
for pont in pontuacoes:
    
    # Extrai os registros para uma pontuação
    df_temp_linhagens = df_linhagens[df_linhagens['risco']==pont]

    # Se o comprimento for maior que 500 coletamos uma amostra de apenas 500
    if (len(df_temp_linhagens) > 5000):
        df_temp_linhagens = df_temp_linhagens.sample(5000, replace = False)
        
    df_temp.append(df_temp_linhagens)    

In [ ]:
len(df_temp) 

In [ ]:
# Converte para dataframe
df_linhagens_bal = pd.concat(df_temp)

In [ ]:
# Total de registros por classe
print(df_linhagens_bal['risco'].value_counts())

In [ ]:
# Plot
df_linhagens_bal['risco'].hist(color = "magenta")
plt.suptitle('Histograma da Pontuação do Risco de Descarte' )
plt.xlabel('Pontuação')
plt.ylabel('Número de Classificações')
plt.show()

As classes ainda estão debalanceadas.

<TABLE>
<TR><TD>Pontuação Atual</TD><TD>Nova Pontuação</TD></TR>
<TR><TD>3,4,5</TD><TD>3</TD></TR>
<TR><TD>6</TD><TD>6</TD></TR>
<TR><TD>7,8,9</TD><TD>9</TD></TR>
</TABLE>

In [ ]:
# Criando as novas categorias

# Classificação mais baixa
df_linhagens_nivel_1 = df_linhagens[df_linhagens['risco'].isin([3,4,5])]
df_linhagens_nivel_1['risco'] = 3

# Classificação média
df_linhagens_nivel_2 = df_linhagens[df_linhagens['risco']==6]
df_linhagens_nivel_2['risco'] = 6

# Classificação mais alta
df_linhagens_nivel_3 = df_linhagens[df_linhagens['risco'].isin([7,8,9])]
df_linhagens_nivel_3['risco'] = 9

# Final
df_linhagens_final = pd.concat([df_linhagens_nivel_1, df_linhagens_nivel_2, df_linhagens_nivel_3], ignore_index = True)

In [ ]:
# Total de registros por classe
print(df_linhagens_final['risco'].value_counts())

In [ ]:
# Plot
df_vinhos_final['quality'].hist(color = "green")
plt.suptitle('Histograma')
plt.xlabel('Pontuação')
plt.ylabel('Número de Classificações')
plt.show()

In [ ]:
# Salvamos uma cópia do noovo dataset em disco
df_linhagens_final.to_csv('dados/df_linhagens_final.csv', index = False)

## Pré-Processamento

In [ ]:
# Separa os atributos da variável target
atributos = [ft for ft in list(df_linhagens_final) if ft not in ['risco']]
atributos

In [ ]:
# Converte a variável target em tipo categórico
df_linhagens_final['risco'] = pd.Categorical(df_linhagens_final['risco'])

In [ ]:
# Cria uma nova variável target com os valores 0, 1 e 2 representam as 3 classificações possíveis
df_linhagens_final['risco_class'] = df_linhagens_final['risco'].cat.codes

In [ ]:
# Nova variável target
target = 'risco_class'

In [ ]:
# Grupos da variável target
grupos_saida = len(set(df_linhagens_final[target]))
grupos_saida

In [ ]:
print(set(df_linhagens_final[target]))

In [ ]:
# Divisão em Dados de Treino e Teste
X_treino, X_teste, y_treino, y_teste = train_test_split(df_linhagens_final[atributos],
                                                        df_linhagens_final[target], 
                                                        test_size = 0.3, 
                                                        random_state = 1)

In [ ]:
print(("Total de X_treino: %i, Total de X_teste: %i, Total de y_treino: %i, Total de y_teste: %i") % (len(X_treino),
                                                                                                      len(X_teste),
                                                                                                      len(y_treino),
                                                                                                      len(y_teste)))

## Construção, Treinamento e Avaliação do Modelo

In [ ]:
# Criação o modelo
modelo_linhagens = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 100, random_state = 100)

Treinamento do modelo.

In [ ]:
%%time
modelo_linhagens.fit(X_treino[atributos], y_treino)

In [ ]:
# Salva o modelo
with open('modelo/modelo_linhagens.p', 'wb') as f:
    pickle.dump(modelo_linhagens, f, 2)

In [ ]:
# Carrega o modelo para avaliação
modelo_linhagens_final = pickle.load(open("modelo/modelo_linhagens.p", "rb" ))

## Avaliação do Modelo

In [ ]:
# Previsões com o modelo treinado
previsoes = modelo_linhagens_final.predict_proba(X_teste)

In [ ]:
previsoes.shape

In [ ]:
# Visualiza todas as previsões
previsoes

In [ ]:
# Visualiza uma previsão do risco no índice zero
previsoes[0]

In [ ]:
list(df_linhagens_final['risco'].cat.categories)

In [ ]:
# Maior previsão feita para o risco no índice zero
print('Argmax: %i' % np.argmax(previsoes[0]))

In [ ]:
# Classificação do risco no índice zero
print('Classificação do Risco: %i' % list(df_linhagens_final['risco'].cat.categories)[np.argmax(previsoes[0])])

In [ ]:
# Agora extraímos a maior probabilidade de cada previsão para calcular a precisão
best_preds = np.asarray([np.argmax(line) for line in previsoes])
print ("Precisão: %0.2f" % precision_score(y_teste, best_preds, average = 'macro'))

In [ ]:
from sklearn.metrics import accuracy_score
print ("Acurácia: %0.2f" % accuracy_score(y_teste, best_preds))

In [ ]:
set(df_linhagens_final['risco'])

In [ ]:
# Função para o plot da Matriz de Confusão
def plot_confusion_matrix(cm, 
                          classes,
                          normalize = False,
                          title = 'Matriz de Confusão',
                          cmap = plt.cm.RdYlGn):
   
    import itertools
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de Confusão Normalizada")
    else:
        print('Matriz de Confusão Não Normalizada')

    print(cm)

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
                 horizontalalignment = "center", 
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label Real')
    plt.xlabel('Label Previsto')

In [ ]:
# Calcula a Matriz de Confusão
cnf_matrix = confusion_matrix(y_teste, best_preds)

In [ ]:
# Plot
plt.figure()
plot_confusion_matrix(cnf_matrix, 
                      classes = list(df_linhagens_final['risco'].cat.categories),
                      title = 'Matriz de Confusão Não Normalizada')
plt.show()

## Previsões com o Modelo Treinado

In [ ]:
# Considerar valores abaixo para uma nova linhagem
temperatura = 34
amonia = 12
idade = 100
peso = 235
sexo = 0
fotoperiodo = 12
ruido = 52
luz = 105
umidade = 45
infeccoes = 0
animais_por_gaiola = 5
linhagem = 0

In [ ]:
# Cria o dataframe com novos dados
novos_dados = pd.DataFrame([[temperatura,
                             amonia,
                             idade,
                             peso,
                             sexo,
                             fotoperiodo,
                             ruido,
                             luz,
                             umidade,
                             infeccoes,
                             animais_por_gaiola,
                             linhagem]], 
                           columns = X_teste.columns.values)

In [ ]:
# Visualiza
novos_dados

In [ ]:
# Previsões com o modelo treinado
previsoes_novos_dados = modelo_linhagens_final.predict_proba((novos_dados))

In [ ]:
# Visualiza
previsoes_novos_dados 

In [ ]:
# Previsão do Risco de Descarte
print(('Previsão do Risco de Descarte: %i') % list(df_linhagens_final['risco'].cat.categories)[np.argmax(previsoes_novos_dados)])

In [ ]:
# Vamos calcular a classificação média para comparar
print(('Classificação Média do Risco: %0.2f') % np.mean(df_linhagens_final['risco'].values))

Deploy do modelo.

# Fim